In [1]:
import numpy as np
import pandas as pd
import cv2 as cv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from skimage.metrics import structural_similarity
from skimage.metrics import mean_squared_error

from main_sklearn import process_set

Read and process a subset of the data to fit into memory

In [2]:
df = pd.read_csv('../data/nyu2_train.csv', header=None)
df = '../' + df
df_sub = df[:300]
df_sub.head()

,0,1
0,../data/nyu2_train/living_room_0038_out/37.jpg,../data/nyu2_train/living_room_0038_out/37.png
1,../data/nyu2_train/living_room_0038_out/115.jpg,../data/nyu2_train/living_room_0038_out/115.png
2,../data/nyu2_train/living_room_0038_out/6.jpg,../data/nyu2_train/living_room_0038_out/6.png
3,../data/nyu2_train/living_room_0038_out/49.jpg,../data/nyu2_train/living_room_0038_out/49.png
4,../data/nyu2_train/living_room_0038_out/152.jpg,../data/nyu2_train/living_room_0038_out/152.png


Split the data into training and testing sets

In [3]:
imgs_train, imgs_test, masks_train, masks_test = train_test_split(df_sub[0], df_sub[1],
                                                                  test_size=0.1,
                                                                  random_state=42)
print(f'Num train images: {len(imgs_train)}')
print(f'Num test images: {len(imgs_test)}')

Num train images: 270
Num test images: 30


Process data (resize and mean-std normalization)

In [4]:
X_train, y_train = process_set(imgs_train, masks_train)
X_test, y_test = process_set(imgs_test, masks_test)
print(f'Input shape: {X_train.shape}')

Input shape: (13547520, 3)


Run cross-validation on different regressors. With more computational resources we could test more models like LinearRegresion, Lasso etc. We choose small numbers of estimators and leafs because they are very computationally demanding

In [6]:
models = {'RF': RandomForestRegressor(n_estimators=5), 
          'GB': GradientBoostingRegressor(n_estimators=10), 
          'KNN': KNeighborsRegressor(leaf_size=5)}
for name, model in models.items():
    kfold = KFold(n_splits=3)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold,
                                 scoring='neg_mean_squared_error')
    print(f'{name}: mse {cv_results.mean()}, std {cv_results.std()}')

RF: mse -0.37465225916419387, std 0.005100912545149864
GB: mse -0.40300856987140526, std 0.009789344752806418
KNN: mse -0.4949546875969517, std 0.04225441882967729


The best model after cross-validation is random forest so we will run a grid search to find the best parameters 

In [7]:
param_grid = {'n_estimators': [2, 5, 10]}
regressor = RandomForestRegressor()
kfold = KFold(n_splits=3)

grid = GridSearchCV(estimator=regressor, param_grid=param_grid,
                    scoring='neg_mean_squared_error', cv=kfold)
grid_result = grid.fit(X_train, y_train)
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print(f'{mean} ({std}) with: {param}')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')

-0.3869587410088702 (0.004850060999379023) with: {'n_estimators': 2}
-0.3745813983867163 (0.005218025816245257) with: {'n_estimators': 5}
-0.3705880908459103 (0.005165461303638868) with: {'n_estimators': 10}
Best: -0.3705880908459103 using {'n_estimators': 10}


After grid-search the best model is the one with 10 estimators. So now we are ready to fit the final model using more data

In [8]:
df_sub = df[:1000]
imgs_train, imgs_test, masks_train, masks_test = train_test_split(df_sub[0], df_sub[1],
                                                                  test_size=0.1,
                                                                  random_state=42)
print(f'Num train images: {len(imgs_train)}')
print(f'Num test images: {len(imgs_test)}')

Num train images: 900
Num test images: 100


In [10]:
X_train, y_train = process_set(imgs_train, masks_train)
X_test, y_test = process_set(imgs_test, masks_test)
print(f'Input shape: {X_train.shape}')

Input shape: (45158400, 3)


In [11]:
regressor = RandomForestRegressor(n_estimators=10, verbose=2)
regressor.fit(X_train, y_train)

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


RandomForestRegressor(n_estimators=10, verbose=2)

Now let's evaluate the best model on test data

In [12]:
y_pred = regressor.predict(X_test)
# Evaluate the model
ssim = structural_similarity(y_test, y_pred, data_range=np.max(y_pred) - np.min(y_pred))
print(f'SSIM: {ssim}')
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

SSIM: 0.4499800386323313
Mean Squared Error: 0.23902518472956327
